In [90]:
import pymongo
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [22]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.australianBushfireDB
bushfiresbyState = db.bushfiresbyState
historicalFires = db.historicalFires

In [23]:
byStateurl = 'https://en.wikipedia.org/wiki/List_of_fires_and_impacts_of_the_2019-20_Australian_bushfire_season'
historicalurl = 'https://en.wikipedia.org/wiki/List_of_major_bushfires_in_Australia'

In [24]:
historicalresponse = requests.get(historicalurl)

In [25]:
soup = BeautifulSoup(historicalresponse.text, 'html.parser')

In [77]:
table = soup.find('table', class_="wikitable")
header_row = soup.find('tr')
table_headers = header_row.find_all("th")
# table_headers = table_head.find_all('th', class_="headerSort")
table_body = table.find("tbody")

In [82]:
# table_headers = table_head.find('tr')
headers = []
for table_header in table_headers:
    try:
        value = table_header.text
        value = value.split("\n")
        headers.append(value[0])
        
    except AttributeError as e:
        print(e)
        
print(headers)
print(len(headers))

['Date', 'Name or description', 'State(s) /territories', 'Area burned (approx.)', 'Fatalities', 'Properties damaged', 'Notes']
7


In [86]:
table_row = table_body.find_all('tr')

In [89]:
datarow = []
data = []
for row in table_row:
    table_data = row.find
    try:
        value = row.text
#         value = value.split("\n")
        data.append(value)
        
    except AttributeError as e:
        print(e)
        
print(row)

['\nDate\n\nName or description\n\nState(s) /territories\n\nArea burned (approx.)\n\nFatalities\n\nProperties damaged\n\nNotes\n', '\nha\n\nacres\n\nHomes(destroyed)\n\nOther buildings\n\nOther damage\n', '\n6 February 1851\n\nBlack Thursday bushfires\n\nVictoria\n\n5,000,000\n\n12,000,000\n\napprox. 12\n\nNil\n\nNil\n\n1 million sheepthousands of cattle\n\n[7][8]\n', '\n1 February 1898\n\nRed Tuesday bushfires\n\nVictoria\n\n260,000\n\n640,000\n\n12\n\nNil\n\n2,000\n\n\n\n[8][9][10]\n', '\nFebruary – March 1926\n\n1926 bushfires\n\nVictoria\n\n390,000\n\n960,000\n\n60\n\n1,000\n\nNil\n\n\n\n[11]\n', '\nDecember 1938 – January 1939\n\nBlack Friday bushfires\n\nVictoria\n\n2,000,000\n\n4,900,000\n\n71\n\n3,700\n\nNil\n\n\n\n[5]\n', '\n14 January – 14 February 1944\n\n1944 Victorian bushfires\n\nVictoria\n\n1,000,000\n\n2,500,000\n\n15–20\n\napprox. 500\n\nNil\n\n\n\n[8]\n', '\n18 November 1944\n\n1944 Blue Mountains bushfire\n\nNew South Wales\n\n\n\n\n\nNil\n\napprox. 40\n\nNil\n\n\n\n

In [91]:
tables = pd.read_html(historicalurl)
tables

ValueError: invalid literal for int() with base 10: '2!data-sort-type="date"'